--- Day 8: Handheld Halting ---

Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (acc, jmp, or nop) and an argument (a signed number like +4 or -20).

* acc increases or decreases a single global value called the accumulator by the value given in the argument. For example, acc +7 would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.
* jmp jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 would cause the instruction 20 lines above to be executed next.
* nop stands for No OPeration - it does nothing. The instruction immediately below it is executed next.

For example, consider the following program:

```text
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```

These instructions are visited in this order:

```text
nop +0  | 1
acc +1  | 2, 8(!)
jmp +4  | 3
acc +3  | 6
jmp -3  | 7
acc -99 |
acc +1  | 4
jmp -4  | 5
acc +6  |
```

First, the nop +0 does nothing. Then, the accumulator is increased from 0 to 1 (acc +1) and jmp +4 sets the next instruction to the other acc +1 near the bottom. After it increases the accumulator from 1 to 2, jmp -4 executes, setting the next instruction to the only acc +3. It sets the accumulator to 5, and jmp -3 causes the program to continue back at the first acc +1.

This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, you know it will never terminate.

Immediately before the program would run an instruction a second time, the value in the accumulator is 5.

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?


In [1]:
def execute_line(line, idx, accumulator):
    action, value = line.split()
    value = int(value)
    return {
        'nop': [idx + 1, accumulator],
        'acc': [idx + 1, accumulator + value],
        'jmp': [idx + value, accumulator]
    }.get(action)


In [2]:
def run_program(filename):
    accumulator = 0
    idx = 0
    executed_lines = []
    with open(filename) as program:
        lines = program.readlines()
    while(idx < len(lines)):
        if idx in executed_lines:
            break
        executed_lines.append(idx)
        idx, accumulator = execute_line(lines[idx], idx, accumulator)
    else:
        print('Finished')
    print(f'current index: {idx}\n{executed_lines}')
    return accumulator

In [3]:
run_program('input.txt')

current index: 364
[0, 1, 2, 3, 4, 113, 241, 364, 365, 386, 387, 388, 389, 176, 177, 178, 179, 370, 371, 372, 373, 516, 517, 561, 562, 394, 395, 489, 11, 12, 13, 14, 15, 96, 97, 98, 99, 414, 415, 416, 417, 83, 84, 85, 86, 292, 293, 294, 78, 79, 80, 541, 542, 543, 544, 525, 609, 610, 611, 612, 88, 89, 90, 91, 92, 67, 166, 167, 168, 575, 576, 577, 156, 157, 158, 159, 483, 484, 485, 486, 487, 116, 117, 243, 244, 245, 246, 44, 452, 453, 546, 547, 7, 8, 335, 34, 35, 123, 124, 125, 126, 127, 300, 301, 302, 303, 304, 257, 258, 259, 498, 499, 500, 279, 280, 495, 357, 358, 359, 421, 422, 147, 148, 149, 150, 151, 571, 602, 603, 604, 462, 463, 464, 617, 618, 619, 620, 621, 189, 190, 191, 192, 193, 597, 598, 599, 296, 297, 298, 551, 355, 136, 137, 227, 219, 220, 221, 512, 513, 514, 410, 434, 435, 436, 437, 438, 20, 21, 375, 376, 347, 348, 349, 350, 181, 182, 183, 104, 105, 106, 107, 342, 196, 197, 198, 199, 519, 520, 521, 251, 252, 253]


1179

--- Part Two ---

After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a jmp is supposed to be a nop, or a nop is supposed to be a jmp. (No acc instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one jmp or nop, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:

```text
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```

If you change the first instruction from nop +0 to jmp +0, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the jmp instructions, the program will still eventually find another jmp instruction and loop forever.

However, if you change the second-to-last instruction (from jmp -4 to nop -4), the program terminates! The instructions are visited in this order:

```text
nop +0  | 1
acc +1  | 2
jmp +4  | 3
acc +3  |
jmp -3  |
acc -99 |
acc +1  | 4
nop -4  | 5
acc +6  | 6
```

After the last instruction (acc +6), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value 8 (acc +1, acc +1, acc +6).

Fix the program so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). What is the value of the accumulator after the program terminates?


In [4]:
import re

In [5]:
def debug_line(line, idx, accumulator):
    action, value = line.split()
    value = int(value)
    result = {
        'nop': [idx + 1, accumulator],
        'acc': [idx + 1, accumulator + value],
        'jmp': [idx + value, accumulator]
    }.get(action)
    result.append(line.rstrip())
    return result

In [6]:
debug_line('nop +111', 0, 0)

[1, 0, 'nop +111']

In [7]:
def debug_program(lines):
    accumulator = 0
    idx = 0
    executed_lines = {}
    finished = False
    while(idx < len(lines)):
        if idx in executed_lines.keys():
            break
        executed_lines[idx] = lines[idx].rstrip()
        idx, accumulator, action = debug_line(lines[idx], idx, accumulator)
    else:
        finished = True
    return finished, accumulator, executed_lines

In [8]:
with open('input.txt') as prog:
    lines = prog.readlines()
finished, total, op_path = debug_program(lines)

In [9]:
total

1179

In [10]:
op_path

{0: 'nop +283',
 1: 'acc +26',
 2: 'acc +37',
 3: 'acc +6',
 4: 'jmp +109',
 113: 'jmp +128',
 241: 'jmp +123',
 364: 'acc +25',
 365: 'jmp +21',
 386: 'nop +57',
 387: 'acc -16',
 388: 'acc +25',
 389: 'jmp -213',
 176: 'acc +3',
 177: 'jmp +1',
 178: 'acc +34',
 179: 'jmp +191',
 370: 'acc +24',
 371: 'acc +23',
 372: 'acc +13',
 373: 'jmp +143',
 516: 'acc -14',
 517: 'jmp +44',
 561: 'acc +44',
 562: 'jmp -168',
 394: 'acc +42',
 395: 'jmp +94',
 489: 'jmp -478',
 11: 'acc -7',
 12: 'acc +27',
 13: 'nop +7',
 14: 'acc +0',
 15: 'jmp +81',
 96: 'nop +145',
 97: 'acc +4',
 98: 'acc +28',
 99: 'jmp +315',
 414: 'acc +22',
 415: 'acc +9',
 416: 'nop +92',
 417: 'jmp -334',
 83: 'acc -9',
 84: 'nop +199',
 85: 'acc +15',
 86: 'jmp +206',
 292: 'acc -10',
 293: 'nop -189',
 294: 'jmp -216',
 78: 'acc -6',
 79: 'acc +42',
 80: 'jmp +461',
 541: 'acc +24',
 542: 'acc +30',
 543: 'acc +14',
 544: 'jmp -19',
 525: 'jmp +84',
 609: 'acc +42',
 610: 'nop -318',
 611: 'acc +36',
 612: 'jmp -524

In [11]:
def match_line(line):
    action, count = line.rstrip().split()
    return {
        'nop':'jmp '+count,
        'jmp':'nop '+count,
        'acc':'acc '+count
    }.get(action)

In [12]:
def fix_program(lines):
    fixed = False
    idx = 0
    while not fixed:
        new_lines = lines.copy()
        new_lines[idx] = match_line(new_lines[idx])
        fixed, total, op_path = debug_program(new_lines)
        idx += 1
    return fixed, idx, total, op_path


In [13]:
new_lines = lines.copy()
fixed, idx, total, op_path = fix_program(new_lines)

In [14]:
fixed

True

In [15]:
idx

305

In [16]:
lines[idx]

'nop -36\n'

In [17]:
total

1089

In [18]:
op_path

{0: 'nop +283',
 1: 'acc +26',
 2: 'acc +37',
 3: 'acc +6',
 4: 'jmp +109',
 113: 'jmp +128',
 241: 'jmp +123',
 364: 'acc +25',
 365: 'jmp +21',
 386: 'nop +57',
 387: 'acc -16',
 388: 'acc +25',
 389: 'jmp -213',
 176: 'acc +3',
 177: 'jmp +1',
 178: 'acc +34',
 179: 'jmp +191',
 370: 'acc +24',
 371: 'acc +23',
 372: 'acc +13',
 373: 'jmp +143',
 516: 'acc -14',
 517: 'jmp +44',
 561: 'acc +44',
 562: 'jmp -168',
 394: 'acc +42',
 395: 'jmp +94',
 489: 'jmp -478',
 11: 'acc -7',
 12: 'acc +27',
 13: 'nop +7',
 14: 'acc +0',
 15: 'jmp +81',
 96: 'nop +145',
 97: 'acc +4',
 98: 'acc +28',
 99: 'jmp +315',
 414: 'acc +22',
 415: 'acc +9',
 416: 'nop +92',
 417: 'jmp -334',
 83: 'acc -9',
 84: 'nop +199',
 85: 'acc +15',
 86: 'jmp +206',
 292: 'acc -10',
 293: 'nop -189',
 294: 'jmp -216',
 78: 'acc -6',
 79: 'acc +42',
 80: 'jmp +461',
 541: 'acc +24',
 542: 'acc +30',
 543: 'acc +14',
 544: 'jmp -19',
 525: 'jmp +84',
 609: 'acc +42',
 610: 'nop -318',
 611: 'acc +36',
 612: 'jmp -524